# Goal: end to end inference and evaluation

given a csv, make predictions and evaluate predictions, then return results in a csv

In [7]:
import pandas as pd

# Load the data
data_path = "../make_test_csv/test.csv"  # Adjust the CSV file path as necessary

try:
    df = pd.read_csv(data_path)
except UnicodeDecodeError:
    df = pd.read_csv(data_path, encoding='ISO-8859-1')

# Drop rows where 'tag_description' is NaN and reset the index
df = df.dropna(subset=['tag_description']).reset_index(drop=True)

# Preserve df_org
df_org = df.copy()

# Print the column names of df_org
print("Columns in df_org:")
print(df_org.columns.tolist())

selected_columns = ['thing', 'property', 'tag_description']
df[selected_columns] = df[selected_columns].astype("string")


Columns in df_org:
['thing', 'property', 'ships_idx', 'tag_name', 'equip_type_code', 'tag_description', 'tx_period', 'tx_type', 'on_change_yn', 'scaling_const', 'signal_type', 'min', 'max', 'data_type', 'description', 'updated_time', 'status_code', 'is_timeout', 'p_thing', 'p_property', 'p_thing_correct', 'p_property_correct', 'MDM', 'pattern']


/tmp/ipykernel_1594509/4046731478.py:7: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [8]:
from datasets import Dataset

def process_df(df):
    output_list = [{
        'translation': {
            'ships_idx': row['ships_idx'],
            'tag_description': row['tag_description'],
            'thing_property': f"<THING_START>{row['thing']}<THING_END><PROPERTY_START>{row['property']}<PROPERTY_END>",
            'answer_thing': f"{row['thing']}",
            'answer_property':f"{row['property']}",
        }
    } for _, row in df.iterrows()]

    return output_list

# Process the DataFrame
processed_data = process_df(df)

# Create a Dataset object
test_dataset = Dataset.from_list(processed_data)

# Print the number of items in the dataset
print(f"The test_dataset contains {len(test_dataset)} items.")


The test_dataset contains 59058 items.


In [9]:
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm

model_checkpoint = "train_tp_checkpoint_1/checkpoint-2240"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base", return_tensors="pt")
# Define additional special tokens
additional_special_tokens = ["<THING_START>", "<THING_END>", "<PROPERTY_START>", "<PROPERTY_END>"]
# Add the additional special tokens to the tokenizer
tokenizer.add_special_tokens({"additional_special_tokens": additional_special_tokens})
# tokenizer.add_special_tokens({'sep_token': "<SEP>"})


pipe = pipeline("translation_XX_to_YY", model=model_checkpoint, tokenizer=tokenizer, return_tensors=True, max_length=64, device=0)

# check what token-ids the special tokens are
# tokenizer.encode("<THING_START><THING_END><PROPERTY_START><PROPERTY_END>")

def extract_seq(tokens, start_value, end_value):
    if start_value not in tokens or end_value not in tokens:
        return None  # Or handle this case according to your requirements
    start_id = tokens.index(start_value)
    end_id = tokens.index(end_value)

    return tokens[start_id+1:end_id]

# problem, what if end tokens are not in?
def process_tensor_output(output):
    tokens = output[0]['translation_token_ids'].tolist()
    thing_seq = extract_seq(tokens, 32100, 32101) # 32100 = <THING_START>, 32101 = <THING_END>
    property_seq = extract_seq(tokens, 32102, 32103) # 32102 = <PROPERTY_START>, 32103 = <PROPERTY_END>
    p_thing = None
    p_property = None
    if (thing_seq is not None):
        p_thing =  tokenizer.decode(thing_seq)
    if (property_seq is not None):
        p_property =  tokenizer.decode(property_seq)
    return p_thing, p_property

In [10]:
p_thing_list = []
p_property_list = []
print("making inference on test set")
for out in tqdm(pipe(KeyDataset(test_dataset["translation"], "tag_description"), batch_size=256)):
    p_thing, p_property = process_tensor_output(out)
    p_thing_list.append(p_thing)
    p_property_list.append(p_property)
print("inference done")

making inference on test set


59058it [01:44, 566.57it/s]                    

inference done


In [11]:
answer_thing = [item['answer_thing'] for item in test_dataset["translation"]]
answer_property = [item['answer_property'] for item in test_dataset["translation"]]

def correctness_test(input, reference):
    assert(len(input) == len(reference))
    correctness_list = []
    for i in range(len(input)):
        correctness_list.append(input[i] == reference[i])
    return correctness_list

# Compare with answer to evaluate correctness
thing_correctness = correctness_test(p_thing_list, answer_thing)
property_correctness = correctness_test(p_property_list, answer_property)

# Calculate accuracy
thing_accuracy = sum(thing_correctness) / len(thing_correctness)
property_accuracy = sum(property_correctness) / len(property_correctness)

# Count True/False values
thing_true_count = thing_correctness.count(True)
thing_false_count = thing_correctness.count(False)
property_true_count = property_correctness.count(True)
property_false_count = property_correctness.count(False)

# Print results
print("Thing prediction accuracy:", thing_accuracy)
print(f"Correct thing predictions: {thing_true_count}, Incorrect thing predictions: {thing_false_count}")
print("Property prediction accuracy:", property_accuracy)
print(f"Correct property predictions: {property_true_count}, Incorrect property predictions: {property_false_count}")

# Create a DataFrame with the results
dict = {
    'p_thing': p_thing_list,
    'p_property': p_property_list,
    'p_thing_correct': thing_correctness,
    'p_property_correct': property_correctness
}
df_pred = pd.DataFrame(dict)


Thing prediction accuracy: 0.15538961698669104
Correct thing predictions: 9177, Incorrect thing predictions: 49881
Property prediction accuracy: 0.14441735243320125
Correct property predictions: 8529, Incorrect property predictions: 50529


In [12]:
# Create a DataFrame with the results
df_pred = pd.DataFrame({
    'p_thing': p_thing_list,
    'p_property': p_property_list,
    'p_thing_correct': thing_correctness,
    'p_property_correct': property_correctness
})

# Merge predictions with the original DataFrame (df_org)
df_org['p_thing'] = df_pred['p_thing']
df_org['p_property'] = df_pred['p_property']
df_org['p_thing_correct'] = df_pred['p_thing_correct']
df_org['p_property_correct'] = df_pred['p_property_correct']

# Save the updated DataFrame to a new CSV file
output_path = "../evaluation/test_with_predictions.csv"
df_org.to_csv(output_path, index=False)

print(f"Updated DataFrame with predictions saved to {output_path}")

Updated DataFrame with predictions saved to ../evaluation/test_with_predictions.csv
